### The main objective for this notebook is to load THINGS1 and THINGS2 raw eeg data so that having a better idea of THINGS eeg preprocessing, which then can be used to train the encoding model on THINGS EEG2 and to test the model on THINGS EEG1.

In [1]:
import os
import pandas as pd

### 1. Load THINGS EEG1 subject meta data

In [2]:
# Load THINGS EEG1 subject1 metadata
subj = 1
project_dir = os.path.join('D:/','UG','Research','Dream_Lab','Dream_Decoding','project_directory')
TH1_dir = os.path.join(project_dir,'eeg_dataset','wake_data','THINGS_EEG1','raw_data','sub-'+format(subj,'02'))
dfcsv = pd.read_csv(os.path.join(TH1_dir,'eeg','sub-'+format(subj,'02')+'_task-rsvp_events.csv'))
dftsv = pd.read_csv(os.path.join(TH1_dir,'eeg','sub-'+format(subj,'02')+'_task-rsvp_events.tsv'), delimiter='\t')

In [3]:
# Load the column names of metadata csv
dfcsv.columns

Index(['eventnumber', 'objectnumber', 'object', 'sequencenumber',
       'presentationnumber', 'blocksequencenumber', 'withinsequencenumber',
       'stimnumber', 'stim', 'istarget', 'stimname', 'response', 'rt',
       'correct', 'time_stimon', 'time_stimoff', 'stimdur'],
      dtype='object')

In [4]:
# Load the column names of metadata tsv
dftsv.columns

Index(['onset', 'duration', 'eventnumber', 'objectnumber', 'object',
       'sequencenumber', 'presentationnumber', 'blocksequencenumber',
       'withinsequencenumber', 'stimnumber', 'stim', 'istarget', 'stimname',
       'response', 'rt', 'correct', 'time_stimon', 'time_stimoff', 'stimdur',
       'sample'],
      dtype='object')

In [5]:
# Select only training 22248 images
dftsv = dftsv[:22248]
# Check the number of training events
print('The number of training images', len(dftsv))

The number of training images 22248


In [6]:
# Select events relevant information
dftsv = dftsv[['onset','object','objectnumber','presentationnumber','sequencenumber','stimname']] 
# Check the column names 
dftsv.columns

Index(['onset', 'object', 'objectnumber', 'presentationnumber',
       'sequencenumber', 'stimname'],
      dtype='object')

### 2. Load THINGS EEG1 subject EEG data

In [7]:
import mne
# Load THINGS EEG1 subject1 raw eeg data
TH1_EEG_dir = os.path.join(TH1_dir,'eeg','sub-'+format(subj,'02')+'_task-rsvp_eeg.vhdr')
raw1 = mne.io.read_raw_brainvision(TH1_EEG_dir, preload=True)

Extracting parameters from D:/UG\Research\Dream_Lab\Dream_Decoding\project_directory\eeg_dataset\wake_data\THINGS_EEG1\raw_data\sub-01\eeg\sub-01_task-rsvp_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 3035739  =      0.000 ...  3035.739 secs...


In [8]:
raw1.info

<Info | 8 non-empty values
 bads: []
 ch_names: Fp1, Fz, F3, F7, FT9, FC5, FC1, C3, T7, TP9, CP5, CP1, Pz, P3, ...
 chs: 63 EEG
 custom_ref_applied: False
 dig: 66 items (3 Cardinal, 63 EEG)
 highpass: 0.0 Hz
 lowpass: 280.0 Hz
 meas_date: 2019-02-22 14:39:19 UTC
 nchan: 63
 projs: []
 sfreq: 1000.0 Hz
>

#### 1) Pick up occipital channels

In [9]:
print(raw1.info['ch_names'])

['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'AF8', 'AF4', 'F2', 'FCz']


In [10]:
if subj in [49, 50]:
    raw1 = raw1.pick(raw1.info['ch_names'][:63])
else:
    pass
# Double check channel names
print(raw1.info['ch_names'])

['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'AF8', 'AF4', 'F2', 'FCz']


In [11]:
# Pick up occipital channels
import numpy as np

chan_idx = np.asarray(mne.pick_channels_regexp(raw1.info['ch_names'],'^O *|^P *'))
print('Picked occipital channels are:')
print(chan_idx)
new_chans = [raw1.info['ch_names'][c] for c in chan_idx]
print(new_chans)
print('The total number of picked occipital channels: ', len(new_chans))

Picked occipital channels are:
[12 13 14 15 16 17 18 19 42 43 44 45 46 47 48 49 50]
['Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2']
The total number of picked occipital channels:  17


In [12]:
# Pick occipital channels
raw1.pick(new_chans)

<RawBrainVision | sub-01_task-rsvp_eeg.eeg, 17 x 3035740 (3035.7 s), ~393.8 MB, data loaded>

#### 2) Create annotations

In [13]:
onset = dftsv['onset'] # in seconds
duration = [0.05]*len(dftsv) # in seconds, too

In [14]:
# Create annotations
my_annot = mne.Annotations(onset=onset, duration=duration, description=['images']*len(dftsv))
# Check annotations
print(my_annot)

<Annotations | 22248 segments: images (22248)>


In [15]:
raw1.set_annotations(my_annot)
print(raw1.annotations)

<Annotations | 22248 segments: images (22248)>


In [16]:
# Create events
events, event_ids = mne.events_from_annotations(raw1)
# Check events
print(event_ids) # The first column is the idx of samples in raw data

Used Annotations descriptions: ['images']
{'images': 10001}


#### 3) Re-reference

In [17]:
# Re-reference raw1 'average'
raw1.set_eeg_reference()

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


<RawBrainVision | sub-01_task-rsvp_eeg.eeg, 17 x 3035740 (3035.7 s), ~393.8 MB, data loaded>

#### 4) Bandpass filter

In [18]:
raw1.filter(l_freq=0.1, h_freq=100, n_jobs=-1)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 33001 samples (33.001 s)



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done  14 out of  17 | elapsed:    6.8s remaining:    1.4s
[Parallel(n_jobs=-1)]: Done  17 out of  17 | elapsed:    7.0s finished


<RawBrainVision | sub-01_task-rsvp_eeg.eeg, 17 x 3035740 (3035.7 s), ~393.8 MB, data loaded>

#### 5) Epoching

In [19]:
epochs = mne.Epochs(raw1, events, tmin=-.2, tmax=.8, baseline=(None,0), preload=True)
del raw1

Not setting metadata
22248 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 22248 events and 1001 original time points ...
0 bad epochs dropped


#### 6) Resampling

In [20]:
re_sfreq = 100
epochs.resample(re_sfreq)
# Check epochs info
epochs.info

<Info | 9 non-empty values
 bads: []
 ch_names: Pz, P3, P7, O1, Oz, O2, P4, P8, P1, P5, PO7, PO3, POz, PO4, PO8, ...
 chs: 17 EEG
 custom_ref_applied: True
 dig: 66 items (3 Cardinal, 63 EEG)
 highpass: 0.1 Hz
 lowpass: 50.0 Hz
 meas_date: 2019-02-22 14:39:19 UTC
 nchan: 17
 projs: []
 sfreq: 100.0 Hz
>

#### 7) Get epoched data and sort data according to images

In [21]:
epoched_data = epochs.get_data()
print(epoched_data.shape)

(22248, 17, 100)


In [22]:
# THINGS EEG2 test images directory
test_img_dir = os.path.join(project_dir,'eeg_dataset','wake_data',
                             'THINGS_EEG2','image_set', 'test_images')
# Create list of test images
test_imgs = os.listdir(test_img_dir)

In [23]:
print(len(test_imgs))

200


In [24]:
# The test THINGS EEG1 data
new_data = []
# Iterate over test images
for test_img in test_imgs:
    # Get the indices of test image in THINGS EEG1 
    indices = dftsv.index[dftsv['object'] == test_img[6:]]
    # Get the data of test image in THINGS EEG1 
    data = [epoched_data[i, :, :] for i in indices]
    # Convert list to array
    data = np.array(data)
    # Average within the test image
    data = np.mean(data, axis=0)
    # Add the data to the test THINGS EEG1 data
    new_data.append(data)
    del data
# Convert list to array
new_data = np.array(new_data)
print(new_data.shape)

(200, 17, 100)


#### 8) Get channel names and times

In [25]:
ch_names1 = epochs.info['ch_names']
times1 = epochs.times

In [26]:
print(ch_names1)

['Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2']


In [27]:
print(times1)

[-0.2  -0.19 -0.18 -0.17 -0.16 -0.15 -0.14 -0.13 -0.12 -0.11 -0.1  -0.09
 -0.08 -0.07 -0.06 -0.05 -0.04 -0.03 -0.02 -0.01  0.    0.01  0.02  0.03
  0.04  0.05  0.06  0.07  0.08  0.09  0.1   0.11  0.12  0.13  0.14  0.15
  0.16  0.17  0.18  0.19  0.2   0.21  0.22  0.23  0.24  0.25  0.26  0.27
  0.28  0.29  0.3   0.31  0.32  0.33  0.34  0.35  0.36  0.37  0.38  0.39
  0.4   0.41  0.42  0.43  0.44  0.45  0.46  0.47  0.48  0.49  0.5   0.51
  0.52  0.53  0.54  0.55  0.56  0.57  0.58  0.59  0.6   0.61  0.62  0.63
  0.64  0.65  0.66  0.67  0.68  0.69  0.7   0.71  0.72  0.73  0.74  0.75
  0.76  0.77  0.78  0.79]


### 3. Load THINGS EEG2 subject1 ses1 training raw data

In [28]:
# Load THINGS EEG2 subject1 metadata
TH2_dir = os.path.join(project_dir, 'eeg_dataset', 'wake_data',
                       'THINGS_EEG2','raw_data','sub-01','ses-01','raw_eeg_training.npy')

In [29]:
eeg_data = np.load(TH2_dir, allow_pickle=True).item()
ch_names = eeg_data['ch_names']
sfreq = eeg_data['sfreq']
ch_types = eeg_data['ch_types']
eeg_data = eeg_data['raw_eeg_data']
# Convert to MNE raw format
info = mne.create_info(ch_names, sfreq, ch_types)
raw2 = mne.io.RawArray(eeg_data, info)

Creating RawArray with float64 data, n_channels=64, n_times=5450560
    Range : 0 ... 5450559 =      0.000 ...  5450.559 secs
Ready.


In [30]:
raw2.info

<Info | 7 non-empty values
 bads: []
 ch_names: Fp1, F3, F7, FT9, FC5, FC1, C3, T7, TP9, CP5, CP1, Pz, P3, P7, ...
 chs: 63 EEG, 1 Stimulus
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 500.0 Hz
 meas_date: unspecified
 nchan: 64
 projs: []
 sfreq: 1000.0 Hz
>

In [31]:
events = mne.find_events(raw2, stim_channel='stim')
# Reject the target trials (event 99999)
idx_target = np.where(events[:,2] == 99999)[0]
events = np.delete(events, idx_target, 0)
print(events)

16800 events found
Event IDs: [    1     2     3 ... 16529 16530 99999]
[[  11446       0   11742]
 [  11646       0    8221]
 [  11845       0    5879]
 ...
 [5433522       0    2634]
 [5433722       0    5362]
 [5433921       0   12636]]


In [32]:
# Select channels
chan_idx = np.asarray(mne.pick_channels_regexp(raw2.info['ch_names'],
    '^O *|^P *'))
new_chans = [raw2.info['ch_names'][c] for c in chan_idx]
raw2.pick_channels(new_chans)

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


<RawArray | 17 x 5450560 (5450.6 s), ~707.0 MB, data loaded>

In [33]:
### Epoching, baseline correction and resampling ###
epochs2 = mne.Epochs(raw2, events, tmin=-.2, tmax=.8, baseline=(None,0), 
                    preload=True)
del raw2
epochs2.info

Not setting metadata
16710 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 16710 events and 1001 original time points ...
0 bad epochs dropped


<Info | 7 non-empty values
 bads: []
 ch_names: Pz, P3, P7, O1, Oz, O2, P4, P8, P1, P5, PO7, PO3, POz, PO4, PO8, ...
 chs: 17 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 500.0 Hz
 meas_date: unspecified
 nchan: 17
 projs: []
 sfreq: 1000.0 Hz
>

In [34]:
epochs2.resample(re_sfreq)
epochs2.info

<Info | 7 non-empty values
 bads: []
 ch_names: Pz, P3, P7, O1, Oz, O2, P4, P8, P1, P5, PO7, PO3, POz, PO4, PO8, ...
 chs: 17 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 50.0 Hz
 meas_date: unspecified
 nchan: 17
 projs: []
 sfreq: 100.0 Hz
>

In [35]:
ch_names2 = epochs2.info['ch_names']
times2 = epochs2.times

In [36]:
print(ch_names2)

['Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2']


In [37]:
print(times2)

[-0.2  -0.19 -0.18 -0.17 -0.16 -0.15 -0.14 -0.13 -0.12 -0.11 -0.1  -0.09
 -0.08 -0.07 -0.06 -0.05 -0.04 -0.03 -0.02 -0.01  0.    0.01  0.02  0.03
  0.04  0.05  0.06  0.07  0.08  0.09  0.1   0.11  0.12  0.13  0.14  0.15
  0.16  0.17  0.18  0.19  0.2   0.21  0.22  0.23  0.24  0.25  0.26  0.27
  0.28  0.29  0.3   0.31  0.32  0.33  0.34  0.35  0.36  0.37  0.38  0.39
  0.4   0.41  0.42  0.43  0.44  0.45  0.46  0.47  0.48  0.49  0.5   0.51
  0.52  0.53  0.54  0.55  0.56  0.57  0.58  0.59  0.6   0.61  0.62  0.63
  0.64  0.65  0.66  0.67  0.68  0.69  0.7   0.71  0.72  0.73  0.74  0.75
  0.76  0.77  0.78  0.79]


In [38]:
# Check the channels sequence of THINGS1 and THINGS2
print(ch_names1)
print(ch_names2)

['Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2']
['Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2']
